#  Line Estimation and Curvature
* This notebook illustrates how to fit polyline to right and left road line data.
* The cell below contains the line class.  Make sure to compile this class before
  running the main program below.  

# Lane Class
* The Lane class holds information about the right and left lane boundaries. 

In [18]:

from scipy.signal import argrelextrema
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
#export PYTHONPATH=/usr/local/lib/python2.7/site-packages:$PYTHONPATH
from scipy.signal import savgol_filter

'''
The Lane class encapulates line/lane properties and includes
logic to find histogram peaks and their Full Width and Half Max
values to use in segmenting the lane
'''

class Lane:
    
    smoothed = []
    imclip = []
    lane = []
 
    
    def __init__(self, binary, xcoords, ycoords, offset, default_value):
      
      self.binary = binary
      self.xcoords = xcoords
      self.ycoords = ycoords
      self.default_value = default_value
      self.offset = offset
        
      #clip the image to between the x and y coordinates supplied
      #these usually correpond to the image stripe and left/right
      self.imclip = binary[ycoords[0]:ycoords[1], xcoords[0]:xcoords[1]]
      histogram = np.sum(self.imclip, axis=0)
      h,w = np.shape(self.imclip)
    
      #use the sav_gol filter to smooth the peaks
      #prior to peak finding
      self.smoothed = savgol_filter(histogram,65,1)
 
      self.lane = self.makeLane( self.smoothed)
 
      
    def makeLane(self, histo):
      '''
      This funcitons determines the peaks and half max
      for the supplied histogram
      '''
      
      peak_indx = np.argmax(histo)
      peak_half_max = histo[peak_indx]/2
   
      starts = np.where(histo[:peak_indx]<peak_half_max)[0]
      stops = np.where(histo[peak_indx:]<peak_half_max)[0]
    
      if np.size(peak_indx)==0:
        peak_indx = self.default_value
      if peak_indx == 0:
        peak_indx = self.default_value
        
      start = starts[-1] if len(starts)>0 else peak_indx-100
      stop = stops[0]+peak_indx if len(stops)>0 else peak_indx+100

        
      return histo, start, stop, peak_indx 
 
    def returnLane(self):
      return self.lane
                       
    def returnClip(self):
      return self.imclip
    
    def returnHistogram(self):
      return self.smoothed
    
    def setOffset(self):
      return self.offset

  

## Video Processing Pipeline.  
* Run the code below to generate output movies
Movies will be stored in ./polyfits under the same name as the source movie

In [ ]:
from moviepy.editor import VideoFileClip

import numpy as np
import glob
import edge_utils as eu

import pickle

from moviepy.editor import *
import cv2

src_dir = './birdseye_edge_test_images/'
out_dir = './polyfits/'


#comment and uncomment the line below to select different
#movies.  Be careful because file output sizes are large
#and can take 10 minutes to write

movies = glob.glob(src_dir+'project_video.mp4')
#movies = glob.glob(src_dir+'challenge_video.mp4')
#movies = glob.glob(src_dir+'harder_challenge_video.mp4')
#movies = glob.glob(src_dir+'*.mp4')


def measure_curvature_pixels(ypoints, xfit):
     
    '''
    Define y-values for the radius of curvature
    Use a maximum y-value, corresponding to the bottom of the image
    Note: this was taken from the instructional material
    '''
    
    y_eval = np.max(ypoints)

    curverad = ((1 + (2*xfit[0]*y_eval + 
                           xfit[1])**2)**1.5) / np.absolute(2*xfit[0])

     
    return curverad

def computeEMD(object1, object2):
    '''
    This function returns the cross entropy and earth
    movers distance between tho histograms.  The histograms
    are retrieved from the Lane object. Lane is shown below.
    '''
    #Lane.returnLane()[0] returns the histogram of counts
    #along the columns of the image.  We retrieve two
    #histograms for calling Shannon cross entropy
    lane1_histo = object1.returnLane()[0]
    lane2_histo = object2.returnLane()[0]
    
    #important to turn histograms into proper statistical distributions
    #divide by total number of activated pixels
    lane1_histo /= np.sum(lane1_histo)
    lane2_histo /= np.sum(lane2_histo)
    
    print("Lane 1 entropy {}".
          format(stats.entropy(lane1_histo)))
    print("Lane 2 entropy {}".
          format(stats.entropy(lane2_histo)))
    print("Lane 1, Lane 2 cross entropy {}".
          format(stats.entropy(lane1_histo,lane2_histo)))

    #I was not able to use the wassertstein distance (aka EMD)
    #on the platform.  It worked on my home computer
    #print("Lane 1, Lnae 2 Earth Mover Distance {}".
    #      format(stats.wasserstein_distance(lane1_histo,lane2_histo)))
    
    ##for the future detemine a rule to stop further sampling
    continue_processing = True
    
    return continue_processing

def anotateImage(image, *argv):
  '''
  Final process before writing or displaying an image
  This function is where the polylines are drawn along
  with their datapoint and associated text  
  '''
  #Below should be rewritten but it works for POC.
  #there are 8 plotting lines (outside lines, center line
  #and mean for left and right.   I've hard code thes values
  #for now

  plot_indices = [0,1,2,3,4,5,6,7]
  offsets = [0,640,0,640,0,640,0,640]
  cvals = [(0,255,0), (0,255,0),
           (0,0,255), (0,0,255),
           (0,255,255), (0,255,255),
           (255,255,0), (255,255,0)]
  plotloc = 100  #y axis for text output 

  #loop over line data and compute polylines
  for indx, arg in enumerate(argv):
    #print(arg)
    if (len(arg)>0):
     
      y = [row[0] for row in arg]
      x = [row[1]+offsets[indx] for row in arg]
   
      #format x and y.  x should be reversed 
      #xback = x[::-1] 
      xfit = np.polyfit(y,x, deg=2)       
      z = np.poly1d(xfit)
        
      curverad = measure_curvature_pixels(yp, xfit)
      #print("curverad {}".format(curverad))
      fitted_points = np.array([[+int(z(y)),int(y)] for y in yp])

      cv2.polylines(image, [fitted_points], False, cvals[indx], 2)
        
      cv2.putText(image, "radius in pix: {}".format(round(curverad,2)), 
                  (500, plotloc), cv2.FONT_HERSHEY_COMPLEX_SMALL, 
                   1.0, cvals[indx])
      #plot the data points along side the polylines
      for i, pt in enumerate(x):
        # print(x[pt])
        cv2.rectangle(image, 
                      (x[i],y[i]), (x[i]+20,y[i]+20), 
                       cvals[indx], -1)  
      plotloc += 50 #move the plot down
    
  return image
 
def computeXFits(indices):
    '''
    Simple function to fit lines from points
    '''
    y = [row[0] for row in indices]
    x = [row[1] for row in indices]
    
    #reverse x for polyfit
    xback = x[::-1] 
    xfit = np.polyfit(y,x, deg=2)      
    return xfit

def update_overall_mask(mask, start_inds, stop_inds, side=0):
  '''
  The overall mask is used see how many pixels fall in the 
  current lane.  If a solid majorit are in the mask, we
  skip the re sampling and polyline step
  '''
  #10 points is sufficient for masking
  ypsmall = np.linspace(1, 720, 10) 
  #double the y values to make convex hull
  yps = np.append(ypsmall,ypsmall)
  #reverse yps to make clockwise side of convex hull
  ypsrev = yps[::-1]    

  start_xfit = computeXFits(start_inds)
  stop_xfit = computeXFits(stop_inds) 

  #start is for the left side of the mask
  #stop is for the right side of the mask
  #prepare the points for the convex hull
  zstart = np.poly1d(start_xfit)
  zstop = np.poly1d(stop_xfit)

  fitted_starts = np.array([[int(zstart(y)),int(y)] for y in ypsmall])
  fitted_stops = np.array([[int(zstop(y)),int(y)] for y in ypsmall[::-1]])
  points = np.append(fitted_starts, fitted_stops, axis=0)
  
  start_pt = left[0] if side==0 else right[0]
  stop_pt = left[1] if side==0 else right[1]
  
  ##create the convex hull                                                                    
  filler = cv2.convexHull(np.array(points), True, True)
  #fill up the convex hull to create a mask
  mask = cv2.drawContours(overall_mask[:,start_pt:stop_pt], [filler],
                                  -1, (255 , 255 , 255),thickness=cv2.FILLED)     


def returnLineIndices(binary_warped, side=0, height=720):
    
    '''
    This function creates a set of polylines by splitting the 
    image into stripes (e.g., 0:100 across the right and
    left side of the image).  Todo:  stop processing when
    the line appears good.  
    '''
    
    #the indices and the four key line componts FWHM-, peak, FWHM+ and mean
    #side is setup to determine left (0) or right (1)
    start_inds,stop_inds,peak_inds,mean_inds = [],[],[],[]
    start_pt = left[0] if side==0 else right[0]
    stop_pt = left[1] if side==0 else right[1]
          
    #offset keeps track of left (offset 0) or right (offset 640)
    #for now its hardcoded but should be made more general
    offset = 0 if side==0 else 640
    
    #the global_lane covers the entire height of the image on the right and left
    global_lane = Lane(binary_warped, (start_pt, stop_pt ), (0,height), 
                          offset = offset, default_value=400)
    
    #this is the fall back number in case we start from nothing.
    lane_default = global_lane.returnLane()[3]    
    
    #windows march through the stripes of the image.  2 is for top botton. 
    #4 is for quarters.  Note that 2,4 would yield 6 unique stripes
    
    windows = [2,3]
 
    for window in windows:  
      for strip in range(window):
         winsize = height//window
         strip_start = strip*winsize
         strip_stop = (strip*winsize) + winsize

         strip_midpt = (strip_start+strip_stop)//2

         #create Lane object to hold points and line sample data
         lane = Lane(binary_warped, (start_pt, stop_pt), (strip_start, strip_stop), 
                           offset=offset, default_value=lane_default)
       
         histo, start, stop, peak_indx  = lane.returnLane()
       
         #accumulate line points 
         stop_inds.append([strip_midpt,stop])
         start_inds.append([strip_midpt,start])
         stop_inds.append([strip_midpt,stop]) 
         peak_inds.append([strip_midpt,peak_indx]) 
         mean_inds.append([strip_midpt,start//3+stop//3+peak_indx//3])
            
    return start_inds, stop_inds, peak_inds, mean_inds

    
def process_image(image):

  '''
  This is the iterator function that processes images thru moviepy
  The function compares the edge image to a mask based on the 
  convex hull of the most recently accepted left and right lines.
  If the percentage of active pixels is high (mean good match),
  then a new poly line is not calculated'''
  
  global left_poly_update_status, right_poly_update_status
  global left_start_inds, left_stop_inds 
  global right_start_inds, right_stop_inds
  global left_peak_inds, right_peak_inds
  global left_mean_inds, right_mean_inds

  binary_warped = image[:,:,0]//255
  #print(np.shape(image))
  h,w = np.shape(binary_warped)
  
  
  noverlapping = np.sum(cv2.bitwise_and(overall_mask[:,:640], 
                                       binary_warped[:,:640]))
    
  percent_overlapping = noverlapping/np.sum(binary_warped[:,:w//2])
  print("left overlapping pixels {}; percent of total {}".
        format(noverlapping, percent_overlapping))

  #decide whether to create a new polyline here if low overlap
  #or low # of pixels
  if percent_overlapping < 0.8 or 100 < noverlapping < 400:
    left_poly_update_status = True
  elif noverlapping < 50:
    left_poly_update_status = False
  
  noverlapping = np.sum(cv2.bitwise_and(overall_mask[:,640:], 
                                       binary_warped[:,640:]))
    
  percent_overlapping = noverlapping/np.sum(binary_warped[:,640:])
  print("right overlapping pixels {}; percent of total {}".
        format(noverlapping, percent_overlapping))
  
  #compare the right size to percent overlap and minimum pixel counts, 
  #as above.
  if percent_overlapping < 0.8 or 100 < noverlapping < 400:
    right_poly_update_status = True
  elif noverlapping < 50:
    right_poly_update_status = False
    
  #if updates are needed, do them here
  if left_poly_update_status:    
    left_start_inds,left_stop_inds,left_peak_inds,left_mean_inds = returnLineIndices(binary_warped, side=0, height=h)
    update_overall_mask(overall_mask, left_start_inds, left_stop_inds, side=0)

  if right_poly_update_status:    
    right_start_inds,right_stop_inds,right_peak_inds,right_mean_inds = returnLineIndices(binary_warped, side=1, height=h)  
    update_overall_mask(overall_mask, right_start_inds, right_stop_inds, side=1)    

  #plt.subplot(3,1,1)
  #plt.imshow(overall_mask)
  #plt.subplot(3,1,2)
  #update_overall_mask(right_start_inds, right_stop_inds)  
  #plt.imshow(overall_mask)
  
  #annotate the image on every frame
  if len(left_peak_inds)>0:
      outimage = anotateImage(image, left_peak_inds, right_peak_inds, 
                              left_start_inds, right_start_inds,
                              left_stop_inds, right_stop_inds,
                              left_mean_inds, right_mean_inds)  
      image = outimage
 
  return image

src_dir = './birdseye_edge_test_images/'
out_dir = './polyfits/'
left_start_inds,left_stop_inds,left_peak_inds,left_mean_inds = [],[],[],[]
right_start_inds,right_stop_inds,right_peak_inds,right_mean_inds = [],[],[],[]

left_poly_update_status = False
right_poly_update_status = False

movies = glob.glob(src_dir+'harder_challenge_video.mp4')
#yp is global; it only needs to be calculated once
yp = np.linspace(1, 720, 100)
overall_mask = np.zeros([720,1280], np.uint8)

#ToDo: determine left and right side with histo
left = [0,640]
right = [640, 1280]

##Loop over movies in the src directory

for movie in movies:
   print(movie)
   outfile = movie.replace(src_dir,out_dir)
   
   input_clip = VideoFileClip(movie)
   start = input_clip.duration/2
   #input_clip = input_clip.subclip(1,4)
   
   #print(type(process_image))
   clip = input_clip.fl_image(process_image) 
   clip.write_videofile(outfile, audio=False)





In [ ]:

from scipy import stats
import glob
import cv2
import numpy as np

def plotLane(lane):
    clip = lane.returnClip()
    #histogram = lane.returnHistogram()
    histogram, start, stop, peak = lane.returnLane() 
    fig = plt.figure() # create figure & 1 axis
   
    ax1 = plt.subplot(211)
 
    ax1.imshow(clip*255, cmap='gray')  
    ax1.autoscale(enable=True, axis='both', tight=True)
    #ax1 = plt.axes([0, 0.6, 0, 1])   
    ax2 = plt.subplot(212, sharex = ax1) 
   # ax2 = plt.axes([0, 0, 0, 1])
    ax2.autoscale(enable=True, axis='both', tight=True)
    
    bins = np.arange(0,len(histogram))

    #print(left_stop)
    plt.plot(bins, histogram)
    plt.ylim([0,histogram[peak] + histogram[peak]//10])
    #plt.show()
    plt.plot(start,histogram[start], 'r.', markersize=25)
    plt.plot(stop,histogram[stop], 'c.', markersize=25)
    plt.plot(peak,histogram[peak], 'c.', markersize=25)   
    
    outname = image.replace('./birdseye_edge_test_images/',
                            './polyfits/'+str(window)+"histo"+str(strip)) 
    
    fig.savefig(outname) # save the figure to file
    plt.close(fig) # close the figure   

def computeEMD(object1, object2):
    lane1_histo = object1.returnLane()[0]
    lane2_histo = object2.returnLane()[0]
    
    #important to turn histograms into proper statistical distributions
    #divide by total number of activated pixels
    lane1_histo /= np.sum(lane1_histo)
    lane2_histo /= np.sum(lane2_histo)
    
    print("Lane 1 entropy {}".format(stats.entropy(lane1_histo)))
    print("Lane 2 entropy {}".format(stats.entropy(lane2_histo)))
    print("Lane 1, Lane 2 cross entropy {}".
           format(stats.entropy(lane1_histo,lane2_histo)))

    #print("Lane 1, Lnae 2 Earth Mover Distance {}".
    #      format(stats.wasserstein_distance(lane1_histo,lane2_histo)))
 
    ymax = np.max([lane1_histo, lane2_histo])
    
    plt.subplot(2,1,1)
    plt.plot(lane1_histo)
    plt.ylim([0,ymax])
    plt.subplot(2,1,1)
    plt.plot(lane2_histo)
    plt.ylim([0,ymax])

    ##for the future detemine a rule to stop further sampling
    continue_processing = True
    
    return continue_processing

def measure_curvature_pixels(ypoints, xfit):
     
    # Define y-value where we want radius of curvature
    # We'll choose the maximum y-value, corresponding to the bottom of the image
    #ypoints = ypoints[::-1]
    y_eval = np.max(ypoints)


    ##### TO-DO: Implement the calculation of R_curve (radius of curvature) #####
 
    curverad = ((1 + (2*xfit[0]*y_eval + 
                           xfit[1])**2)**1.5) / np.absolute(2*xfit[0])

     
    return curverad


def display_lane(binary_warped, *argv):
  out_image = np.dstack((binary_warped, binary_warped, binary_warped))
  yp = np.linspace(0, 720, 10)
    
  fig, ax = plt.subplots( nrows=1, ncols=1 )
  plt.imshow(out_image*255)    
  cvals = ['g', 'g', 'r', 'r', 'c', 'c', 'y', 'y']
  for indx, arg in enumerate(argv):
    
    #print(arg)
    if (len(arg)>0):
      y = [row[0] for row in arg]
      x = [row[1] for row in arg]
      x = x[::-1] 
      xfit = np.polyfit(y,x, deg=2)      
      z = np.poly1d(xfit)
 
      plt.plot(z(yp), yp, cvals[indx], linewidth=2)
      plt.xlim([0,1280]) 
      plt.show()
    
      curverad = measure_curvature_pixels(yp, xfit)
      print("curvrad {}".format(curverad))
  plt.xlim([0,1280]) 
  outname = image.replace('./birdseye_edge_test_images/', './polyfits/'+str(window))
  #fig.savefig(outname) # save the figure to file
  #plt.close(fig) # close the figure
  print(outname)
  #cv2.imwrite(outname, out_image)
 # plt.show()

    
left_start_inds = []
left_stop_inds = []
left_peak_inds = []
left_mean_inds = []

right_start_inds = []
right_stop_inds = []
right_peak_inds = []
right_mean_inds = []

#images = glob.glob('./birdseye_test_images/test1.pgm')
#images = glob.glob('./birdseye_edge_test_images/straight_lines2.jpg')

images = glob.glob('./birdseye_edge_test_images/test2.jpg')
print(len(images))

if len(images)==0:
    print("********--NO IMAGE FOUND--********")

for image in images:
  print(image)    
  binary_warped = cv2.imread(image,-1)//255

    
  h,w = np.shape(binary_warped)
  print("w {}".format(w))
  print("h {}".format(h))
    
  global_left_lane = Lane(binary_warped, (0, w//2), (0,h), offset = 0, default_value=200)
  global_right_lane = Lane(binary_warped, (w//2, w), (0,h), offset = 0, default_value=640-200)
  #plotLane(global_left_lane)
  computeEMD(global_left_lane, global_left_lane)
  #plotLane(global_lane)
  left_default = global_left_lane.returnLane()[3] 
  right_default = global_right_lane.returnLane()[3]

  print("default values ")
    
  windows = [1,2,3]
  #use a for loop here
  for window in windows:
    
    for strip in range(window):
       winsize = h//window
       strip_start = strip*winsize
       strip_stop = (strip*winsize) + winsize

       strip_midpt = (strip_start+strip_stop)//2

       left = Lane(binary_warped, (0, w//2), (strip_start, strip_stop), 
                         offset=0, default_value=left_default)
       
       plotLane(left)
       lhisto, lstart, lstop, lpeak_indx  = left.returnLane()
       
       left_stop_inds.append([strip_midpt,lstop])
       #print(start,stop,peak_indx)

       left_start_inds.append([strip_midpt,lstart])
       left_stop_inds.append([strip_midpt,lstop]) 
       left_peak_inds.append([strip_midpt,lpeak_indx]) 
       left_mean_inds.append([strip_midpt,lstart//3+lstop//3+lpeak_indx//3])
        
       right = Lane(binary_warped, (w//2,w), (strip_start, strip_stop), 
                          offset=640, default_value = right_default)
       #plotLane(right)
        
       rhisto, rstart, rstop, rpeak_indx  = right.returnLane()
 
       right_start_inds.append([strip_midpt,640+rstart])
       right_stop_inds.append([strip_midpt,640+rstop])
       right_peak_inds.append([strip_midpt,640+rpeak_indx]) 
       right_mean_inds.append([strip_midpt,640+(rstart//3+rstop//3+rpeak_indx//3)])      
       #if stop == 0 or peak_indx==0:
            


    #   continue_searching = computeEMD(top_half, bottom_half)
    
    display_lane(binary_warped, left_peak_inds, right_peak_inds, 
                                left_start_inds, right_start_inds,
                                left_stop_inds, right_stop_inds,
                                left_mean_inds, right_mean_inds)
  
    #display_lane(binary_warped, left_mean_inds, right_mean_inds)
  #if continue_searching:
 
    
    
    

## 